# Respostas

## Setup

### Definir imports

In [1]:
from datetime import date, datetime

from core.carteira import Carteira, Posicao
from core.var.var import VarParametrico, VarHistorico
from inputs.data_handler import InputsDataHandler
from core.fatores_risco.exposicao import ExposicaoCarteira
from core.fatores_risco.fatores_risco import MatrizFatoresRisco
from core.renda_fixa.renda_fixa import RendaFixa
from utils.enums import IntervaloConfianca, AcoesBr, AcoesUs, Opcoes, Futuros, Titulos

## Instanciar gerenciador de inputs

In [2]:
data_handler = InputsDataHandler()

## Definição da carteira do grupo

## Questão 1